## INITIALIZE REQUIREMENTS

### Load Dependencies

In [1]:
%matplotlib inline
# !pip install python-dotenv
# load_dotenv()
# from dotenv import load_dotenv

import os
import glob
import warnings 
import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from rasterio.merge import merge
from rasterio.plot import show

warnings.filterwarnings("ignore")

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_minimum
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.areaofinterest import define_area

from IPython.display import clear_output

ModuleNotFoundError: No module named 'dotenv'

### G-Drive Dependencies

In [2]:
import os.path
import google.auth

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account

# If modifying these scopes, delete the file credentials.
SCOPES = ["https://www.googleapis.com/auth/drive"]
credential_path = '../creds/credentials_2.json'
u_credential_path = '../creds/token.json'

In [3]:
# root = os.getcwd()

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [4]:
dc = datacube.Datacube(app="Radar_water_detection")

## G-DRIVE

### G-Drive Folder IDs

In [5]:
FLOOD_MEAN_ID = "196YsHy1SXjNDnja6LjVhhlvoIt-Jg91l"
FLOOD_MEDIAN_ID = "1Qhum99pKi1Qyon8hcp4K8S_DGaJh5a5e"
PREFLOOD_MEAN_ID = "1K0KqGlLxdUCsXg4771k_uf_x2UwbHlqa"
PREFLOOD_MEDIAN_ID = "1Ovu5Q58xZRGpKsvowVr49klOrgVoZ0XQ"

FLOOD_MEAN_TEST_ID = "1bv76i244wyJzfpWN57wM7Yh4jQ38gove"
PREFLOOD_MEAN_TEST_ID = "1msS6VuX_8UptluKIuQG5PJNOrwqYtacY"

### G-Drive Function Definitions

In [6]:
def create_user_token():
    creds = None
    if os.path.exists(u_credential_path):
        creds = Credentials.from_authorized_user_file(u_credential_path, SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credential_path, SCOPES)
            creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(u_credential_path, "w") as token:
        token.write(creds.to_json())
    return creds
    
def create_token():
    ''''
        credential: provide the json creditials you would get from google service.
    '''
    creds = None
    creds = service_account.Credentials.from_service_account_file(credential_path, scopes=SCOPES)
    return creds

def list_gdrive():
    '''
    List the 10 recent files from the google drive
    '''
    # creds = create_token()
    u_creds = create_user_token()
    try:
        service = build("drive", "v3", credentials=u_creds)
      
        results = (service.files().list(pageSize=20, fields="nextPageToken, files(id, name)").execute())
        items = results.get("files", [])

        if not items:
            print("No files found.")
            return
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
    except HttpError as error:
        print(f"An error occurred: {error}")


def upload_to_gdrive(file_paths, folder_id=None):
    '''
        Uploading files to google drive
    '''
    creds = create_token()

    for f_path in file_paths:
        try:
            # create drive api client
            service = build("drive", "v3", credentials=creds)
            folder_path = '../Supplementary_data/DriveCredentials/{}'.format(folder_id)
            
            f_name = os.path.basename(f_path)
            file_metadata = {"name": f_name, "parents": [folder_id]}
            media = MediaFileUpload(f_path, resumable=True)
            # pylint: disable=maybe-no-member
            
            file = (service.files().create(body=file_metadata, media_body=media).execute())
            print('\033[32m' + "{} UPLOADED SUCCESSFULLY".format(f_name) + '\033[0m')
    
            # DELETE FROM SANDBOX TO SAVE DISC MEMORY
            os.remove(f_path)
        except HttpError as error:
            print(f"An error occurred: {error}")
            file = None
    return

In [ ]:
list_gdrive()

>Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=975171912910-05mf229112maahcr4pn6fvvuhmdt1jlm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52075%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=IKzulGbOARVtq6m0z5BkuoFtp200KP&access_type=offline                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## CUSTOMIZE DATA
In the next cells define the flood dates, add the threshold value and grid vector file

### Define Flooding Dates

In [7]:
# DEFINE PERIODS
#timeframe
timerange = ('2024-02', '2024-09')

pre_flood = ['2024-02', '2024-03', '2024-04'] # 3 MONTHS PRIOR
flood = ['2024-05', '2024-06', '2024-07', '2024-08', '2024-09'] # MONTHS DURING
# post_flood = ['2024-09-10', '2024-10-15']

### Get Threshold from `1.aoi-threshold.ipynb`

In [8]:
th_aoi = -27.395682

### Upload file and calculate centroids

In [9]:
# LOAD FILE FROM SANDBOX
# grid = gpd.read_file("input/Lake Chad.geojson")
grid = gpd.read_file("input/TCD_55KM_4CTEST.geojson")
# grid = gpd.read_file("input/TCD_55KM_BASE.geojson")

# Calculate centroids and store in centroid list c[]. The array c[] will be used to loop all grid cells
c = [] # INIT EMPTY GRID CENTROID
g = grid.centroid # STORE ALL GRID CENTROIDS in g
for i in g:
    c.append([round(i.x, 5), round(i.y, 5)]) # EXTRACT x and y FROM POINT(x,y) AND APPEND TO c[]

# AOI MOSAIC COVERING LAKE CHAD
aoi_m = []
for i in c:
    # aoi_m.append(define_area(i[1], i[0], buffer=0.05))
    aoi_m.append(define_area(i[1], i[0], buffer=0.25))

In [10]:
print(len(aoi_m))

4


## RWD FUNCTION DEFINITIONS

### Speckle Filter Function

In [11]:
#defining a function to apply lee filtering on S1 image 
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    
    return img_output

### Classifier Function

In [12]:
def S1_water_classifier(da, threshold):
    water_data_array = da < threshold
    return water_data_array.to_dataset(name="s1_water")

## Main For Loop Interation

In [13]:
grid_val = 1
e_log =[]

for aoi in aoi_m:
    geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
    geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)
    g = geopolygon_gdf.centroid
    print("\n\n"+ '\033[32m' + "PROCESSING GRID CELL {}/{} CENTROID ({}, {})".format(grid_val, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)) + '\033[0m')

    # Get the latitude and longitude range of the geopolygon
    lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
    lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

    # LOAD SENTINEL-1 DATA
    try:
        S1 = load_ard(dc=dc,
                    products=["s1_rtc"],
                    measurements=['vv', 'vh'],
                    y=lat_range,
                    x=lon_range,
                    time=timerange,
                    output_crs = "EPSG:6933",
                    resolution = (-20,20),
                    group_by="solar_day",
                    dtype='native'
                    )
    except:
        # LOG ERROR aoi CENTROIDS AND CONTINUE LOOPING
        e_log.append([g.y[0], g.x[0], "P"])
        print("\n\n"+ '\033[31m' + "ERROR PROCESSING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(grid_val, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)) + '\033[0m')
        continue
    
    # INIT TIMESTEPS 
    timesteps = [2,4,6,9,11] 

    # The lee filter above doesn't handle null values
    # We therefore set null values to 0 before applying the filter
    valid = np.isfinite(S1)
    S1 = S1.where(valid, 0)

    # Create a new entry in dataset corresponding to filtered VV and VH data
    # S1["filtered_vv"] = S1.vv.groupby("time").apply(lee_filter, size=7)
    S1["filtered_vh"] = S1.vh.groupby("time").apply(lee_filter, size=7)

    # Null pixels should remain null
    # S1['filtered_vv'] = S1.filtered_vv.where(valid.vv)
    S1['filtered_vh'] = S1.filtered_vh.where(valid.vh)   

    # Convert the digital numbers to dB
    # S1['filtered_vv'] = 10 * np.log10(S1.filtered_vv)
    S1['filtered_vh'] = 10 * np.log10(S1.filtered_vh)

    threshold_vh = th_aoi

    S1['water'] = S1_water_classifier(S1.filtered_vh, threshold_vh).s1_water
    FS1 = S1.water
    PRFS1 = S1.water

    
    # CREATE OUTPUTS
    # EXPORT TO RASTERS - UPLOAD TO G-DRIVE - DELETE FROM SANDBOX
    #PREFLOOD
    S1_PreFlood = PRFS1.sel(time = pre_flood, method = 'nearest').mean(dim = 'time')
    preflood_val = 'CELL_' + str(grid_val) + '_PRE_FLOOD_MEAN'
    preflood_name = preflood_val +'.tif'
    preflood_out = 'output/preflood/' + preflood_name
    S1_PreFlood.rio.to_raster(preflood_out)
    #PREFLOOD META
    prf_meta_text = '### Meta Data - GRID CELL ID = ' + str(grid_val) + ' ###' + '\n' 'Time Range: ' + pre_flood[0] + ' - ' + pre_flood[-1] + '\n' + 'Lat Range: ' +  str(lat_range) + ' Lon Range: ' + str(lon_range) + '\n' + 'Coordinate Reference System: ' + str(geopolygon.crs)
    text_flie_name = preflood_val + '_META.txt'
    prf_meta_path = 'output/preflood/' + text_flie_name
    with open(prf_meta_path, mode='w') as f:
         f.write(prf_meta_text)
    try:
        # upload_to_gdrive([preflood_out, prf_meta_path], os.getenv("PREFLOOD_MEAN_ID"))
        upload_to_gdrive([preflood_out, prf_meta_path], PREFLOOD_MEAN_TEST_ID)
    except:
        e_log.append([g.y[0], g.x[0], "U"])
        print("\n\n"+ '\033[31m' + "ERROR UPLOADING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(grid_val, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)) + '\033[0m')
        continue

    
    #FLOOD
    S1_Flood = FS1.sel(time = flood, method = 'nearest').mean(dim = 'time')
    flood_val = 'CELL_' + str(grid_val) + '_FLOOD_MEAN'
    flood_name = flood_val + '.tif'
    flood_out = 'output/flood/' + flood_name
    S1_Flood.rio.to_raster(flood_out)
    #FLOOD META
    f_meta_text = '### Meta Data - GRID CELL ID = ' + str(grid_val) + ' ###' + '\n' 'Time Range: ' + flood[0] + ' - ' + flood[-1] + '\n' + 'Lat Range: ' +  str(lat_range) + ' Lon Range: ' + str(lon_range) + '\n' + 'Coordinate Reference System: ' + str(geopolygon.crs)
    text_flie_name = flood_val + '_META.txt'
    f_meta_path = 'output/flood/' + text_flie_name
    with open(f_meta_path, mode='w') as f:
         f.write(f_meta_text)
    try:
        # upload_to_gdrive([flood_out, f_meta_path], os.getenv("FLOOD_MEAN_ID"))
        upload_to_gdrive([flood_out, f_meta_path], FLOOD_MEAN_TEST_ID)
    except:
        e_log.append([g.y[0], g.x[0], "U"])
        print("\n\n"+ '\033[31m' + "ERROR UPLOADING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(grid_val, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)) + '\033[0m')
        continue

    grid_val += 1
    clear_output()

if len(e_log) == 0:
    print("\n\n"+ '\033[32m' + "GRID PROCESSED AND UPLOADED SUCCESSFULLY" + '\033[0m')




GRID PROCESSED AND UPLOADED SUCCESSFULLY


In [14]:
# PRINT ERROR DATA
print(e_log, len(e_log))

[] 0


## Write Outputs To Disc

In [15]:
dirs = ['flood', 'preflood', 'postflood']

for dir in dirs:
    loc = "output/" + dir
    out = "output/{}/Merged_{}.tif".format(dir, dir)
    extension = "*.tif"
    q = os.path.join(loc, extension)
    files = glob.glob(q)

    r =[]
    for f in files:
        s = rasterio.open(f)
        r.append(s)
    if len(r)>0:
        mosaic, out_trans = merge(r)
        out_meta = s.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans
                    })
        with rasterio.open(out, "w", **out_meta) as dest:
            dest.write(mosaic)
            # upload_to_gdrive(out, "flood")

In [16]:
creds = create_token()
service = build("drive", "v3", credentials=creds)
results = service.files().list(fields = "files(id)", pageSize = 1000).execute()
items = results.get('files')
print(len(items))
storage = service.about().get(fields = "storageQuota").execute()
storage

33


{'storageQuota': {'limit': '16106127360',
  'usage': '24051582',
  'usageInDrive': '24051582',
  'usageInDriveTrash': '0'}}

In [17]:
for item in items:
    try:
        response = service.files().delete(fileId=item['id']).execute()
        print('File Deleted')
    except HttpError as error:
        print(f"An error occurred: {error}")

File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
File Deleted
An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1GAQ4RSKt1s0Da6Uwm2ewv6-u7lYRwNil? returned "The user does not have sufficient permissions for this file.". Details: "[{'message': 'The user does not have sufficient permissions for this file.', 'domain': 'global', 'reason': 'insufficientFilePermissions'}]">
An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1rfvJIJ2NlM3m8mDKq8FXMzrMTgOh6xSC? returned "The user does not have sufficient permissions for this file.". Details: "[{'message': 'The user does not have sufficient permissions for this file.', 'domain': 'global', 'reason': 'insufficientFilePermissions'}]">
An error occurred: <HttpError 403 when